In [2]:
path = "C:/bigcon/"

import pandas as pd
import numpy as np

In [3]:
cheoin = pd.read_excel(path + "input/additional_dataset/3. 산업중분류, 종사자 규모 및 읍면동별 사업체수, 종사자수.xls", sheet_name="처인", header=3,nrows = 2 )
gihung = pd.read_excel(path + "input/additional_dataset/3. 산업중분류, 종사자 규모 및 읍면동별 사업체수, 종사자수.xls", sheet_name="기흥", header=3,nrows = 2)
suji = pd.read_excel(path + "input/additional_dataset/3. 산업중분류, 종사자 규모 및 읍면동별 사업체수, 종사자수.xls", sheet_name="수지", header=3,nrows = 2)






In [4]:
def slicein(df):
    lst = np.array(df.iloc[0]) == '사업체수 Estab.'
    tmp = df.loc[1:,lst]
    return tmp

cheoin = slicein(cheoin)
gihung = slicein(gihung)
suji = slicein(suji)


In [5]:
# 종사자수 전처리
cheoin = cheoin.drop(['용인시', '처인구'], axis=1)
cheoin = cheoin.T

gihung = gihung.drop(['용인시', '기흥구'], axis=1)
gihung = gihung.T

suji = suji.drop(['용인시', '수지구'], axis=1)
suji = suji.T

works = pd.concat([cheoin, gihung, suji], axis=0)
works.columns = ['종사자수']

In [16]:
works = works.reset_index()
works.iloc[3,0] = '남사읍'
works.set_index('index',inplace = True)


In [17]:
# 전기차

electric_car = pd.read_csv(path + 'input/additional_dataset/용인시 전기차 현황.csv', encoding='euc-kr')
electric_car['등록대수'] = electric_car['승용차수'] + electric_car['승합차수'] + electric_car['화물차수'] + electric_car['특수차수']

electric_car = electric_car.drop(['등록연도','연료별','용도별','승용차수', '승합차수', '화물차수', '특수차수'], axis=1)

cars = electric_car.groupby('시군구명').sum()

In [26]:
# 인구수
import re
pop = pd.read_csv(path + "input/additional_dataset/population.csv",encoding = 'CP949',header = 1)
popidx = pop.pop('연령별')
pop.index = list(map(lambda x: re.sub(' ','',x),popidx))


pop_new = pop.copy()
pop_new['인구수'] = pop_new['20대'] + pop_new['30대'] + pop_new['40대'] + pop_new['50대'] + pop_new['60대']
pop_new = pop_new.drop(['20대', '30대', '40대', '50대', '60대','합계'], axis=1)



In [27]:
pop_cheoin = pop_new.iloc[:13,:]
pop_gihung = pop_new.iloc[13:28,:]
pop_suji = pop_new.iloc[28:,:]

In [28]:
pop_cheoin['인구비율'] = pop_cheoin['인구수'] / 188979
pop_gihung['인구비율'] = pop_gihung['인구수'] / 308373
pop_suji['인구비율'] = pop_suji['인구수'] / 261194

pop_cheoin['동별전기차'] = pop_cheoin['인구비율'] * 1826
pop_gihung['동별전기차'] = pop_gihung['인구비율'] * 2946
pop_suji['동별전기차'] = pop_suji['인구비율'] * 1963

C:\Users\ghrbs\AppData\Local\Temp\ipykernel_17596\2498493404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_cheoin['인구비율'] = pop_cheoin['인구수'] / 188979
C:\Users\ghrbs\AppData\Local\Temp\ipykernel_17596\2498493404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_gihung['인구비율'] = pop_gihung['인구수'] / 308373
C:\Users\ghrbs\AppData\Local\Temp\ipykernel_17596\2498493404.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [29]:
pop_total = pd.concat([pop_cheoin, pop_gihung, pop_suji], axis=0)
pop_total.drop(['인구수','인구비율'], axis=1, inplace=True)

In [30]:
df = pd.merge(works, pop_total, left_index=True, right_index=True, how='left')
# NaN 나타나는지 확인 위해 일부러 left join 사용

In [31]:
df

,종사자수,동별전기차
index,,
포곡읍,3795,237.300092
모현읍,4623,189.896115
이동읍,4111,141.719143
남사읍,2578,158.010033
원삼면,1813,56.660603
백암면,2150,58.786341
양지면,3100,137.979776
중앙동,3938,182.378730
역삼동,3434,316.628800


In [32]:
def get_traffic(data):
    data['전기차_교통량'] = data['종사자수'] / data['종사자수'].sum() * data['동별전기차']
    return data

df = get_traffic(df)

In [33]:
df.to_excel(path + '/output/data/checkpoint/전기차 교통량.xlsx')